In [1]:
from GAME.utils.config import config
import numpy as np
import sys
import os
import math
import pandas as pd
import random
import copy

config_data = config()

In [2]:
def parse_weights(weights_file):
    with open(weights_file, 'rb') as f:
        rl_mem_size = 1048576
        weights = np.fromfile(f, dtype=np.double, count=rl_mem_size)
        
        offset = rl_mem_size * 8
        f.seek(offset, os.SEEK_SET)
        m = np.fromfile(f, dtype=np.int_, count=1)

        offset += 4
        f.seek(offset, os.SEEK_SET)
        safe = np.fromfile(f, dtype=np.intc, count=1)

        offset += 4
        f.seek(offset, os.SEEK_SET)
        calls = np.fromfile(f, dtype=np.int_, count=1)

        offset += 4
        f.seek(offset, os.SEEK_SET)
        clearhits = np.fromfile(f, dtype=np.int_, count=1)

        offset += 4
        f.seek(offset, os.SEEK_SET)
        collisions = np.fromfile(f, dtype=np.int_, count=1)

        offset += 4
        f.seek(offset, os.SEEK_SET)
        data = np.fromfile(f, dtype=np.int_, count=m[0])

    return weights, m, safe, calls, clearhits, collisions, data

class collision_table:
    def __init__(self, m, safe, calls, clearhits, collisions, data):
        self.m = m
        self.safe = safe
        self.calls = calls
        self.clearhits = clearhits
        self.collisions = collisions
        self.data = data

def hash_UNH(ints:list, num_ints:int, m:int, increment:int, rndseq:list):
    i = 0
    index = 0
    sum = 0

    for i in range(num_ints):
        index = ints[i]
        index = index + (increment * i)
        index = index % 2048
        while index < 0:
            index = index + 2048
        sum = sum + int(rndseq[int(index)])
    
    index = int(sum % m)
    while index < 0:
        index = index + m
    
    return int(index)
    
def hash(ints:list, num_ints:list, ct:collision_table, rndseq:list):
    j = 0
    ccheck = 0

    ct.calls = ct.calls + 1
    j = hash_UNH(ints, num_ints, ct.m, 449, rndseq)
    ccheck = hash_UNH(ints, num_ints, 2147483647, 457, rndseq)
    if ccheck == ct.data[j]:
        ct.clearhits = ct.clearhits + 1
    elif ct.data[j] == -1:
        ct.clearhits = ct.clearhits + 1
        ct.data[j] = ccheck
    elif ct.safe == 0:
        ct.collisions = ct.collisions + 1
    else:
        h2 = 1 + 2 * hash_UNH(ints, num_ints, int(sys.maxsize / 4), 449, rndseq)
        i = 0
        i += 1
        while i:
            ct.collisions = ct.collisions + 1
            j = int((j + h2) % (ct.m))
            if i > ct.m:
                raise ValueError("Out of memory")
            if ccheck == ct.data[j]:
                break
            if ct.data[j] == -1:
                ct.data[j] = ccheck
                break
            i += 1
    return int(j)

def GetTiles(num_tilings:int, ctable:collision_table, floats:list, num_floats:int, ints:list, num_ints:int, rndseq:list):
    tiles = np.zeros(shape = (1, num_tilings), dtype = int)[0]
    i = 0
    j = 0
    qstate = np.zeros(shape = (1, 20))[0]
    base = np.zeros(shape = (1, 20))[0]
    coordinates = np.zeros(shape = (1, 20 * 2 + 1))[0]
    num_coordinates = num_floats + num_ints + 1

    for i in range(num_ints):
        coordinates[num_floats + 1 + i] = ints[i]

    for i in range(num_floats):
        qstate[i] = int(math.floor(floats[i] * num_tilings))
        base[i] = 0

    for j in range(num_tilings):
        for i in range(num_floats):
            coordinates[i] = qstate[i] - ((qstate[i] - base[i]) % num_tilings)
            base[i] = base[i] + 1 + (2*i)
        
        coordinates[num_floats] = j
        coordinates = coordinates[:4]
        # print(coordinates)
        tiles[j] = hash(coordinates, num_coordinates, ctable, rndseq)

    return tiles

def GetTiles1(nt:int, ct:collision_table, f1:float, h1:int, h2:int, rndseq:list):
    f_tmp_arr = [f1]
    i_tmp_arr = [h1, h2]
    return GetTiles(nt, ct, f_tmp_arr, 1, i_tmp_arr, 2, rndseq)

def GetTilesNoCT(num_tilings:int, floats:list, num_floats:int, ints:list, num_ints:int, rndseq:list):
    tiles = np.zeros(shape = (1, num_tilings), dtype = int)[0]
    i = 0
    j = 0
    qstate = np.zeros(shape = (1, 20))[0]
    base = np.zeros(shape = (1, 20))[0]
    coordinates = np.zeros(shape = (1, 20 * 2 + 1))[0]
    num_coordinates = num_floats + num_ints + 1

    for i in range(num_ints):
        coordinates[num_floats + 1 + i] = ints[i]

    for i in range(num_floats):
        qstate[i] = int(math.floor(floats[i] * num_tilings))
        base[i] = 0

    for j in range(num_tilings):
        for i in range(num_floats):
            coordinates[i] = qstate[i] - ((qstate[i] - base[i]) % num_tilings)
            base[i] = base[i] + 1 + (2*i)
        
        coordinates[num_floats] = j
        coordinates = coordinates[:4]
        # print(coordinates)
        tiles[j] = hash_UNH(coordinates, num_coordinates, 1048576, 449, rndseq)

    return tiles    

In [3]:
rndseq = [1741056371, 1375685356, 701348523, 4076593990, 2093110520, 468248461, 1985621603, 866109850, 1714556343, 827892570, 626853143, 1491689172, 2880622022, 2612286481, 243430465, 557702279, 1894334113, 1105329255, 1040285335, 3940314006, 2402835498, 3970972667, 850345044, 3961051996, 35323459, 4227508794, 4213821926, 87850132, 1977138785, 2314820795, 2828603285, 2985030067, 99612416, 3919657701, 3389770704, 1212638397, 522395304, 477848773, 343104197, 1582004579, 997221412, 295569884, 2866064626, 454078267, 869131088, 1212618076, 3144622617, 3130752501, 199301660, 2133063721, 4171504403, 3049934568, 2553428192, 2035105076, 3160362615, 4207635461, 2894471467, 2853852578, 3195057523, 990141651, 915714991, 3807437982, 1328682313, 4253175465, 141612210, 2317964360, 2584394965, 236496964, 2891707278, 1289170331, 155378950, 3291721677, 2743369517, 2916382279, 3727826156, 1254371574, 539198828, 4222748420, 4109110201, 549095107, 1040577004, 3647419319, 2572198883, 349428020, 4150173938, 279508485, 2483142324, 3158604026, 1231676963, 3491420777, 1786662014, 1361425224, 2220046996, 4214331792, 844580078, 2612849125, 634325237, 3923926611, 1621807794, 3498441300, 3627411668, 1719821464, 3651700597, 1701862026, 1063419945, 1155431589, 2303612761, 3545345452, 2257944812, 400852465, 2349623025, 2092989627, 586998886, 3663792995, 2942483587, 3026792250, 4289668886, 2477524767, 3094155572, 764276559, 2312418915, 901892068, 612591576, 3986035436, 1161364184, 3842701469, 1997132197, 2529293343, 2510979786, 1816768144, 3440797882, 2893719282, 3030960793, 2999072554, 3406352225, 3385032798, 1845700826, 1289185817, 3990016921, 1283033856, 576101076, 416415460, 3654108579, 2445672959, 3375028501, 21180398, 352487265, 2081645214, 1778152065, 3446760870, 1051412943, 1900793402, 1973899126, 3934151935, 2179686909, 4274232167, 3205360012, 2119054013, 4185689194, 3344671804, 49591021, 1914105075, 21884928, 277571535, 2572180247, 2676937880, 174130129, 3165064638, 2613029646, 3583508950, 3857897206, 2101265686, 2384539181, 2936194745, 1674152479, 1889000972, 1451825707, 555417863, 232324491, 381854116, 3822047890, 3533196085, 1641371089, 1815338434, 3170364894, 334488864, 3353521117, 2756542856, 475208430, 3949339724, 997793448, 2902157928, 4082648582, 1194010126, 231926706, 527186491, 3226741419, 3125737718, 3340148339, 285793028, 1612843687, 1295103323, 60820770, 222780877, 2609346646, 3580906652, 2263486360, 4075527251, 229112410, 3635459547, 1354956125, 156904101, 3802397623, 323441818, 828584483, 1322153563, 1970386509, 560670833, 774229815, 2163858530, 2631342512, 510478927, 3514594079, 1256815552, 1513164155, 2620845514, 1539048155, 3051240786, 3059155668, 3293324437, 3365385490, 2150683355, 1642092285, 373540977, 4291448245, 2827618142, 3775476646, 1431780125, 4008718190, 537020801, 3802169336, 3813108195, 297179321, 1100945442, 2823538941, 2718177936, 4098489877, 1263048237, 3893700043, 1552920173, 1162225542, 3846154637, 2322696748, 1973730410, 4005199673, 34957290, 730642055, 459601249, 2870798224, 1536171749, 44568951, 1305338171, 2271245450, 1994093729, 1747593347, 2401100858, 969722516, 1558739294, 2441795294, 1471617503, 411920782, 1776103377, 139941015, 1966723502, 94061409, 2226962453, 1201075102, 1292664165, 2582049538, 3290616780, 2049467375, 1630838118, 3487614803, 2642897124, 492536427, 1728106497, 3871605674, 3872880224, 4292431712, 233686749, 3448582250, 362368562, 4107378012, 3513671095, 845177368, 936933686, 2999490239, 3042745475, 3939364351, 1715220826, 231716574, 1024693615, 1962379179, 3489718914, 3380101502, 3588096447, 2884615697, 737110840, 3189905147, 900426409, 2750567795, 4063753659, 2939566724, 339428287, 704761941, 4047277487, 1587784595, 1326792178, 127034874, 3378165112, 2431647141, 725640277, 1110813491, 1899553487, 3700253227, 2745114026, 830645498, 3696915308, 1232081268, 2960575218, 2876581148, 2903042953, 2504897592, 3989955358, 1401362735, 3784088619, 4239237036, 594124750, 3737447052, 3436516706, 600480272, 2608672495, 88547046, 3552186831, 1756462219, 438000377, 3745809836, 446303805, 1688751360, 645656363, 3272413590, 3357763120, 3571629038, 4246071260, 1517062260, 127316331, 1073638245, 2035978300, 1755435824, 3292094616, 454788888, 829355147, 3682467810, 330321234, 2931406631, 323445883, 3912739757, 1879787147, 667788120, 999809814, 350418984, 2454092352, 4281493266, 2043383394, 37294194, 1163328876, 597738590, 950498636, 2305651739, 3018808242, 566412698, 2387869072, 2349663185, 1660960134, 2701255897, 2824414513, 3348778106, 2309434794, 1794262264, 1161922769, 1317774257, 2363504429, 3793487498, 1965800508, 3305682766, 2967795738, 3594326555, 2124082381, 646938290, 1891032914, 3538279495, 278430934, 2705007697, 522021877, 1518145752, 402433341, 2351230972, 3670953644, 3005936580, 3279395428, 2998842833, 3676358197, 2648051380, 3551785567, 3621870513, 2963889507, 3545311126, 3368155771, 2158971995, 3474166124, 3919520188, 1376983850, 4101430180, 523831539, 2318371154, 4052012401, 857394179, 761098006, 4237349711, 2750314904, 1700019332, 2370074391, 1946249829, 2235029176, 1369619326, 2856444, 3144174431, 1455749051, 422789286, 2025307884, 3089649981, 1596454577, 3383832521, 3703883416, 739374979, 3605937903, 561943961, 911443438, 2077633755, 4246899911, 3552284848, 2153138348, 1748971326, 762195278, 3972196386, 376246673, 1144876097, 1610090546, 3029983796, 4012564567, 1762694550, 2126701930, 2244840859, 983444606, 3684286522, 2445831237, 1632598353, 2035984611, 1782479336, 6181509, 736043110, 2102256728, 3869545079, 1845476814, 1312169927, 2429070074, 103080454, 2184547502, 346821778, 4193839839, 2642237194, 2081741002, 1257804251, 3225212358, 1622723810, 3982004225, 94671870, 2642337338, 4214572151, 1226588563, 349335252, 239377006, 4234629609, 3219253956, 2256454435, 4238433783, 3600903711, 3182408402, 2166466192, 1698823777, 2823768679, 825502903, 3002063534, 2419565926, 3625158037, 3258410820, 1477366973, 1339170551, 577265235, 2643069519, 2045050377, 463695859, 3086256249, 753974660, 4270932557, 1684030598, 3382892902, 769897894, 3350442210, 1461703950, 341673792, 1296417867, 3590756410, 1742520368, 2140772012, 2085311043, 4278330710, 586834998, 1491826341, 809169159, 1099514280, 1711462629, 2607714584, 3118685112, 3867217416, 2071084755, 1588885902, 3999962415, 4091008857, 3652140660, 500141271, 3018362777, 1772003812, 283097198, 2469526913, 2610082190, 1770580034, 942061397, 4131597481, 733758100, 1524857194, 3258046805, 1658246909, 1753713618, 275977288, 2597035408, 264911162, 2256261088, 1581795872, 2032367067, 4182497378, 3546494179, 3611831153, 348651811, 2453298456, 3039623187, 1950561261, 1722337376, 4087399110, 83274203, 1054100562, 2639099183, 2278770748, 4282404724, 2206360042, 707414557, 3389842638, 42838336, 1660261375, 1107832521, 3681945051, 3075755834, 3601867776, 3664649893, 2046980988, 2401091057, 307294293, 1478434356, 2502168396, 2019460943, 246108137, 443387540, 1862341119, 516878385, 518096502, 10791573, 3367625281, 392794150, 350162734, 1302184892, 2664741821, 2729242304, 2373203597, 417407713, 2583962290, 1840437378, 2447880414, 2222168866, 1196875753, 47032719, 1105206362, 3242146651, 1130171825, 266679200, 3437461328, 1461351326, 1884457075, 540082785, 287292370, 1593192097, 1396396642, 562365421, 1770143424, 2628869644, 65437475, 411206953, 2705390847, 925540490, 1710484870, 2029024481, 1924309262, 1660951142, 3868887806, 470755517, 1327807622, 1314656691, 1342061256, 4004424032, 1615556291, 1267214641, 582168638, 3260611346, 2092406986, 346652003, 2813799317, 1708520901, 543393899, 3971128334, 1707887912, 608713376, 13789716, 2095609635, 2678307077, 2024721048, 307430398, 94309482, 1817809552, 2731291042, 2677782043, 2200780323, 3505858632, 2800935096, 1172551242, 2210575855, 474447806, 347103411, 1276563151, 1124921876, 68836522, 222126674, 538484131, 3662844662, 2568139949, 3256115214, 707714670, 1003520575, 3957254648, 457919036, 1676140349, 963785939, 2229829702, 4259403047, 255038794, 51874286, 4134725137, 3174124832, 405626450, 2677155107, 2057660791, 2260639638, 4046774517, 1282073410, 3704247194, 4221680147, 3457705325, 2861207589, 821664950, 3090893993, 1882037948, 2541944435, 1280380231, 4073216704, 3770690955, 1253945466, 3075223919, 2289899768, 2864165953, 968013189, 63098102, 2317792875, 3873764913, 2712054616, 4074555258, 1524658692, 2535868112, 3640350172, 3173110344, 1200141102, 1823039246, 1326081601, 3878599745, 237913509, 1435202861, 2261387588, 3957034034, 1681875152, 3904888375, 3393550513, 1932849793, 1664558776, 3281773897, 1710198096, 3390669358, 1491337024, 2732423020, 586161557, 604968583, 1195130634, 3609547580, 350325982, 4195290451, 4060910484, 3909812492, 4213088543, 783853429, 4108746700, 198314015, 3012951726, 2752184726, 395586048, 2670398618, 1252964984, 1717693786, 1198270035, 1043297010, 3295715432, 2074214034, 891851733, 1419734942, 1294538419, 2315456210, 1865688493, 1670881063, 138383235, 3767342614, 3064589066, 1213900950, 2227063054, 3243761712, 2349194735, 2642220965, 3165005981, 876524522, 3885954096, 4171023996, 1510968091, 1735085043, 1898504718, 2784605026, 2698772436, 447921665, 1353855304, 1324860840, 122733422, 3138281772, 910375644, 1038958301, 3376263651, 4017087551, 555124591, 223090452, 2631565912, 3907224606, 666892900, 2889367925, 1978816612, 1631429506, 1395388768, 1997042964, 3807866058, 3689998338, 2950915676, 463982993, 2569729530, 1738112186, 3295484475, 1972326231, 2277187938, 2886296667, 3972708104, 1904843018, 3012428826, 662754539, 3991807842, 2121709829, 853174238, 3821746640, 2515654662, 2054230061, 4279519015, 2484802177, 154788743, 2963049698, 1995685977, 3204000083, 3925956964, 1919521138, 2111346961, 4088107773, 3849684117, 286357383, 70771139, 805967387, 208699262, 3641831511, 3918096605, 905697818, 1885253506, 1864763764, 1575630733, 4115507201, 3173547927, 963702306, 4183228206, 1373194434, 955729063, 491592570, 944900141, 2695900765, 3635278865, 501363735, 959661450, 227429445, 2240867491, 3725073686, 1512391674, 3597752494, 508346428, 804541288, 664334900, 641694124, 3479588781, 1852621768, 2449916776, 1998263958, 1742131862, 3391487730, 3186697956, 815829247, 115322228, 1265646813, 2013218286, 442270850, 844503293, 2065166137, 135601464, 3048028091, 2363764695, 1433187788, 1811593862, 1446971529, 2501936657, 930433599, 2322036544, 347077536, 2469099496, 2536289282, 661620861, 2928674371, 2978764008, 27142284, 94358553, 1321712098, 4063349385, 1568640132, 3994288796, 2768822102, 2503229078, 3529908952, 116322644, 3064673960, 3691065913, 2159984238, 3535800951, 3370765661, 487388400, 1914095736, 96062907, 1677943616, 76905092, 1396240934, 4177305024, 3865779715, 1997730793, 683369005, 1966074073, 843848247, 3784620853, 3433978820, 2868413584, 1671599322, 3028796637, 1126566584, 1442025863, 1068219936, 1064981771, 682676178, 2840879884, 4143441835, 2125170881, 3516037414, 2232135108, 3060590070, 3862561038, 3956400276, 625188891, 981976761, 844003843, 3790874982, 4292291509, 1125165865, 1252865845, 2115488163, 1533722261, 4052045348, 3553175476, 3176209340, 675705195, 1444779168, 3402356296, 3802131261, 162124795, 773922562, 3930502823, 2547082175, 399845997, 4072541884, 2313421932, 2213521805, 2558298311, 2661796232, 3984535685, 1368605800, 1752159834, 769922743, 3357156171, 164419746, 1231055335, 120745973, 2980018690, 868117403, 759297627, 621548269, 657799216, 17945162, 1899704697, 1839296031, 1088954739, 2794196948, 3439669234, 289529656, 1981311351, 691782298, 2012419044, 2491483093, 1764050960, 2975327359, 1544778094, 1414637258, 1762607506, 1258499266, 4148209291, 3249168427, 3483974528, 2096989, 846318470, 3262205995, 2173812172, 2515111564, 691279851, 3732412077, 558968353, 1882029730, 2655594592, 3145239356, 155715742, 848702043, 3393341097, 3126613723, 2911173661, 2994454352, 3303518591, 1597815657, 2244151223, 1496453668, 1952042286, 3038972259, 2818932826, 2151721541, 660325511, 2558586909, 1140315293, 702988701, 1066322933, 2983483481, 3671634778, 408854336, 3110389585, 3837226535, 2923611352, 696612200, 323050949, 364191471, 2396146342, 820635369, 1303198513, 1688819987, 3961383701, 2741009846, 2715515831, 2426054174, 1827718556, 3668673831, 1505778109, 1790103638, 3485166450, 484977085, 3282334804, 1041986218, 188172005, 3821800764, 2355558902, 415911143, 2243386017, 2692964195, 668186470, 3995734266, 1633148740, 602701999, 1383769706, 1156731593, 2863426378, 801811798, 2640690316, 2412611312, 610612551, 1387656868, 798756723, 2405514553, 229147452, 1915785743, 2505022244, 471930176, 2269808601, 4286414127, 445424297, 3990872826, 2271557369, 2512980267, 413421364, 3328472141, 2935695022, 2024005010, 813644573, 1595742438, 2186207256, 401163055, 2375181139, 4158103717, 3536212811, 1781587354, 2954475279, 919598521, 517329205, 3189007692, 2798165917, 2688566130, 110542192, 3348826743, 3032647402, 2440799151, 1366615311, 2286574383, 318491826, 990848321, 2799874669, 2109531185, 2120883216, 3854548790, 883181244, 4003588865, 2629350359, 3352828269, 3150699064, 2277403142, 679876110, 1171604601, 2055878248, 740387272, 3642597537, 4122545840, 2229922059, 2807239120, 3643268638, 4246902903, 2916016089, 4014580169, 1715563099, 4202171518, 1643481609, 1218173217, 3283566528, 3654760326, 2769706901, 1593925317, 1120411965, 1076738978, 574991210, 3783560100, 1003119636, 2728106823, 2986007569, 4214937150, 49773379, 607577414, 2089857117, 339411279, 107307944, 16707505, 4190946036, 110703113, 2427276469, 3828480864, 3249323477, 3887998189, 596088099, 2256917631, 1368879959, 816210112, 1403811127, 3715209118, 492073732, 311947829, 528963749, 204285021, 3298276852, 1024832657, 3257583775, 2606775992, 3031547079, 1236204648, 911543618, 2184290374, 3361028614, 1794021164, 408931507, 3205131124, 3139975940, 3593956364, 2356760334, 2867811443, 1150187950, 2098256789, 1873234223, 2981405548, 3705565619, 366854050, 1460580353, 4244993089, 2481844240, 4257982061, 1928305699, 2738851712, 486552370, 3740455989, 25507582, 2326478877, 2553294486, 4224975725, 3282014310, 3726632699, 538521086, 201470733, 2271808273, 340471469, 2069644150, 809952243, 3849542339, 2470493875, 1525395814, 3991171956, 1300137314, 3400798021, 273857601, 2560570493, 328089766, 3489552681, 3909783511, 3993127739, 2161220938, 2242678933, 4266448534, 3943371774, 2572395624, 1375572283, 119542517, 576532899, 785313203, 2071808377, 3374256309, 706982852, 2002267336, 1438516061, 3725939201, 1938007201, 1385256397, 262031064, 3176566248, 2051124465, 2581117422, 2528988021, 3533600325, 555411315, 2872852666, 3666318231, 3709894487, 1630226682, 67299738, 3207860113, 3531986932, 2680055627, 4205315540, 798518028, 3119211546, 380376346, 2935928173, 851771141, 1255602665, 2489332878, 138043959, 3536667787, 3098060495, 1421601027, 3114168555, 1668280493, 1172936409, 1254844242, 97094359, 2546885455, 1980309194, 3288911229, 2789107721, 2908337907, 1112394892, 406118173, 4251579540, 928507053, 1610774308, 172335537, 2198518320, 1668359077, 3203412694, 605156129, 2062005937, 1067081375, 2631255207, 442849437, 1863503570, 2213640257, 3786020613, 3138004789, 4074628657, 2000998419, 429177396, 4195537257, 631192744, 2444552818, 1577154585, 211767038, 2083794931, 2029979281, 2495726990, 3549951992, 909353415, 3884661317, 2360499864, 1370265293, 3771056216, 3318147673, 2276452187, 1188975485, 317998329, 2138980108, 815637633, 1245400618, 951237, 3580698204, 470136675, 3825983734, 4280610687, 2737736659, 3425654038, 1789083755, 2999136391, 810541901, 1419489336, 2794466981, 3021874263, 1286548248, 3749719625, 594523349, 853892195, 876654728, 3680551297, 4042204836, 223083610, 4196889305, 2812355530, 282566722, 3170674160, 1012234519, 3040516261, 700402231, 172396804, 1828970004, 2751585972, 3632199573, 2073855415, 4076785575, 946294114, 647469360, 3710530890, 756244178, 339511020, 3145564470, 436727052, 96252734, 604020810, 2537126772, 1672527504, 3273417431, 1508492052, 1715817088, 1278708817, 4097871640, 1076847320, 1759333579, 2366331984, 666773376, 2817233934, 819891836, 370920714, 1532830363, 2357490677, 1656799471, 3407626482, 3680990082, 1376227458, 3860856572, 986449557, 690499765, 2896472590, 1684799023, 2268995938, 2777998840, 2624944770, 2490990286, 1166304110, 2895389528, 953051036, 962972608, 2750161736, 2164801565, 2109447954, 874373241, 2705647437, 3624576272, 3624709137, 1903677972, 1677024740, 4254925178, 1486916748, 3178038622, 2985077641, 4149320143, 1564860879, 3165905695, 2822767781, 3724877622, 2796340067, 432587466, 3043840940, 3203169052, 828173293, 265161911, 252730605, 377234364, 686169921, 2950761316, 1331564557, 1250634107, 3893651703, 3816075250, 2232147867, 2265077679, 3916886168, 1492909223, 1494004899, 2547916415, 4068898773, 4213622656, 832969657, 2842847378, 3898157632, 1411835822, 849105353, 2054179180, 4139795185, 3859378455, 2962018393, 4265208807, 2450860006, 990876781, 2883991077, 1451200844, 1070808613, 3719249293, 1007543866, 1188110808, 3863592737, 1481870851, 969156751, 1153031299, 2367334762, 3387356933, 4091363386, 2925597332, 2865870082, 625149278, 120450635, 3888828276, 3833126573, 1591063377, 3002305376, 1419703551, 1873346964, 3959944179, 887176987, 2869858191, 2255305956, 1122973172, 3804320823, 1598568142, 4046939101, 1045877107, 923832034, 505311652, 2343405774, 2646459007, 2115483357, 1609346129, 604974690, 1711265180, 258244141, 2196754701, 2623134521, 396278165, 3043980052, 1545561472, 781509267, 2478321571, 481153182, 2627906360, 4219957778, 623618266, 2585521910, 978613864, 3898211195, 287972909, 3622816627, 2440473484, 4246576930, 1212677347, 1659623836, 1062208551, 2852102842, 700573440, 2964484929, 673107238, 1013729412, 3007670037, 809611631, 2444596794, 3023631581, 4191935913, 2370890421, 1874385835, 606351320, 1771564441, 3634301033, 1419682569, 3600279248, 1925478911, 342144131, 479211072, 3026065437, 4110857933, 2780706553, 1557725746, 1894318818, 2910642625, 3767879164, 628309209, 3512071878, 1538036480, 1791621574, 2734487826, 3858494610, 2010469207, 1805144976, 210791133, 3848315968, 221724791, 20528803, 892843546, 850046122, 2147484139, 2555739044, 3857023690, 1176898131, 1565182558, 2198340281, 1069077362, 1652563170, 2149436696, 1917435223, 775889268, 1697368002, 1972511225, 2586030810, 3682946109, 90710149, 2246942456, 911953765, 2591152639, 2627846418, 870452173, 96315360, 1123229256, 1530842593, 728488290, 3324561248, 2577424182, 1092700276, 55788041, 283765074, 3708262968, 1164450161, 3555906458, 463141556, 979036796, 2066935934, 2234681237, 477227257, 2021798334, 3930795966, 1023563864, 2622623190, 2901889575, 709076655, 1965901201, 2653784599, 2948388250, 117725252, 112237986, 44726447, 2714490571, 243039107, 2864715080, 3986644892, 1530148450, 948873022, 1044701249, 3998871695, 616979250, 1138144749, 2529834371, 1788157893, 3377866754, 3838787619, 320955393, 2052362398, 451661917, 3246738007, 1350621882, 368738270, 1353179188, 1981896585, 1119587004, 253450794, 103450568, 2949783551, 1392163175, 4097393988, 3760617559, 1204871305, 2339063195, 2258617740, 3377222776, 510949488, 1848760162, 1780328011, 1126081162, 4001502074, 3981972851, 4158259392, 643053637, 3367023926, 3800930380, 2856228845, 1564047179, 3163211177, 3805945305, 3021838811, 1909530682, 2195026020, 1644802060, 1195964836, 2171662397, 4206225372, 2416227652, 558768018, 541052066, 356320887, 1186366605, 4269617792, 3995254248, 3617899879, 2826611913, 3385548009, 187206432, 1704040363, 1192966470, 2573977223, 2374201188, 674548944, 2943916664, 1140220238, 518811268, 2097557444, 560466618, 3553640800, 1420936572, 3851504788, 166071372, 3698170875, 1493794774, 263101233, 396749802, 1982680010, 3725608643, 2710722730, 2036659797, 3532738603, 2614035114, 2124274581, 1120370616, 4106912466, 3670578555, 1542268372, 1377577252, 2910474056, 1230107591, 4006501425, 2514282889, 2792119169, 888274063, 3726795568, 1049449707, 128201808, 4112980196, 4118025611, 1358828791, 1785755807, 1635003967, 2526179540, 532987942, 1760851550, 462307856, 56073043, 1437616832, 549609345, 932036813, 523280958, 4116866141, 1457240945, 1778221420, 1411236010, 3440012013, 3452530180, 1445908854, 1601418324, 3575230763, 4100824157, 1780402879, 998926600, 2496918881, 2623759858, 2436327664, 2854044800, 922135338, 1665697741, 1716620449, 3690310000, 3365851492, 20404114, 1858044696, 3704068114, 3175365664, 2361716466, 360354801, 1883005240, 3660750299, 1777626583, 3018911888, 3095962229, 3437205080, 2743356088, 4275939694, 420128755, 1027919526, 943488747, 745438180, 4129184451, 4260371360, 1952864370, 1124262237, 210260041, 3424645124, 1533996935, 3698027730, 2311492998, 3065062954, 394173530, 2155722637, 84465361, 667342210, 885328144, 813032377, 997736433, 694491968, 3820526436, 894628155, 1522994305, 2397176770, 2802701271, 2193002686, 785362776, 885757975, 2201189304, 2255287264, 872440514, 3798303881, 1918329077, 233878332, 3210908915, 757861040, 1619421670, 4066166310, 1546184766, 2389559040, 3290953170, 396889814, 195217720, 3470059822, 1515328588, 2933617419, 56314257, 3356594829, 948395856, 812459579, 267416541, 3294366750, 2921359965, 4225591551, 967938049, 3257044731, 2867678170, 1517360489, 1636321061, 3847439251, 1940844655, 2337828548, 184469196, 559204299, 1108518300, 2226783973, 1112345383, 2689514259, 4239098504, 82856974, 4010990096, 1738726313, 3028305208, 978001532, 2552865848, 2002734196, 181337102, 3209174190, 1542962882, 2576968525, 430999123, 4238659476, 3446918468, 3524901085, 3887393702, 4245181528, 4279376536, 2877679044, 896210737, 266847964, 1519591725, 2899905171, 2398435979, 4254000124, 2717750348, 2189038007, 3844663797, 263180650, 1123194862, 3433136474, 2528896404, 1254658283, 3307550535, 2672361092, 1910995328, 2521557720, 1031915017, 575169465, 55070029, 282933461, 40901793, 3088786985, 44018329, 3935596840, 372781368, 2157310339, 3711881710, 470401822, 2044772402, 4226178045, 2231735920, 2584254640, 1086974144, 1591362364, 403974708, 518869911, 3440626120, 4180002430, 710733508, 1686533284, 1348297902, 933816911, 3155461338, 47674063, 3929774307, 1616798091, 3125825311, 3603219238, 556324184, 3502221196, 3559614422, 47818220, 339595125, 2603679830, 2169468247, 322469428, 1616088112, 305446374, 1596243042, 4251078161, 2669577787, 1334219216, 3590727657, 1051008671, 4171943691, 3733778750, 2964234413, 284082095, 205908571, 3430624162, 2169736128, 4171849713, 1412365362, 3152900203, 2601523627, 4292369163, 491087849, 3247606594, 3843490526, 3227635476, 2429241163, 3099047508, 3369074631, 2807747269, 2688134753, 3426395057, 1369870097, 123610775, 695067362, 614020844, 1748349712, 2424755504, 3213070731, 3174317326, 3670810850, 726038868, 3009165015, 4117546077, 2725735730, 4248986301, 3337832579, 663065090, 1387455613, 4166898091, 3104342702, 1950747670, 3195619515, 3165288578, 2646672837, 1184352152, 1235948865, 149417154, 4017057891, 3044833907, 4139921330, 1839674378, 1748619183, 3516286746, 1096637514, 1145572403, 3078395501, 4178632943, 3541016896, 3084208671, 236637919, 2954296050, 1935031479, 2655578710, 3296773053, 2443422820, 3008210026, 586058033, 139989177, 1846193121, 1625790719, 797070835, 35172755, 3143497582, 2879117517, 2321743506, 3121564456, 435554682, 3718347020, 600703013, 3870407073, 246419641, 1290307542, 1250256900, 2494835886, 2855610504, 3600913657, 1871978325, 1171846740, 2282098132, 4019497529, 437468846, 3345636465, 2693069174, 630091924, 579856744, 1742781679, 3871982805, 1584271224, 2169316936, 3078863447, 129224237, 1094631780, 3450588212, 2340954993, 1374111663, 1448486871, 2739806298, 3520770776, 2373977551, 3200725899, 1929363453, 2279763672, 3400894496, 196081582, 4246145486, 4055607167, 1001148153, 1937868005, 2151932423, 626122735, 107417362, 386383892, 516547344, 2223673279, 920500394, 1581027294, 2138236325, 3284440265, 749001539, 2879150025, 1698355690, 3295193595, 1164092835, 2654241054, 2791752553, 727135063, 4212038310, 2867949584, 743045872, 2997087223, 93363108, 3306996331]

In [4]:
tile_widthds3v2 = [2.000000, 2.000000, 3.000000, 2.000000, 3.000000, 2.000000, 3.000000, 3.000000, 3.000000, 4.000000, 4.000000, 10.000000, 10.000000]
tile_widthds4v3 = [2.000000, 2.000000, 2.000000, 3.000000, 3.000000, 3.000000, 3.000000, 2.000000, 2.000000, 3.000000, 3.000000, 3.000000, 3.000000, 4.000000, 4.000000, 4.000000, 10.000000, 10.000000, 10.000000]

In [5]:
state_mapping = [0, 12, 9, 1, 4, 4, 4, 1, 11, 9, 9, 9, 9, 4, 9, 9, 9, 11, 9]
action_mapping = [0, 1, 2, 2]

In [5]:
state_mapping = sorted([0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12], reverse=True)
action_mapping = [0, 0, 0, 0]

In [8]:
state_mapping = [11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 0, 1, 2]
action_mapping = [0, 0, 0, 0]

In [9]:
keeper_3v2 = 3
keeper_4v3 = 1

k3v2_logs_folder = os.path.join(config_data['logs_path'], '202212260013-UbuntuXenial-3v2-4hrs')
k3v2_weights_file = os.path.join(k3v2_logs_folder, 'k{}-weights.dat'.format(keeper_3v2))
k3v2_q_file = os.path.join(k3v2_logs_folder, 'player1672042390.txt')

k4v3_weights_folder = os.path.join(config_data['logs_path'], '202211031855-UbuntuXenial-4v3-weights')
k4v3_weights_file = os.path.join(k4v3_weights_folder, 'k{}-weights.dat'.format(keeper_4v3))
weights4v3, m4v3, safe4v3, calls4v3, clearhits4v3, collisions4v3, data4v3 = parse_weights(k4v3_weights_file)
for weight_idx in range(len(weights4v3)):
    weights4v3[weight_idx] = 0
ct4v3 = collision_table(m4v3, 0, 0, 0, 0, data4v3)
for data_idx in range(len(ct4v3.data)):
    ct4v3.data[data_idx] = -1

qdf = pd.read_csv(k3v2_q_file, index_col = False, names=["State Value", "Action", "State", "Tile"])
qdf = qdf.drop_duplicates(
  subset = ["State Value", "Action", "State"],
  keep = 'first').reset_index(drop = True)
qdf = qdf.dropna()

# 3v2
weights3v2, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(k3v2_weights_file)
ct3v2 = collision_table(m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2)
transfer_weights = np.zeros(shape=weights4v3.shape)
num_tilings = 32

print("Transferring weights...")
for row_idx, row in qdf.iterrows():
    state_val, action, state, tile = row['State Value'], row['Action'], row['State'], row['Tile']
    tiles = GetTiles(num_tilings, ct3v2, [state_val / tile_widthds3v2[int(state)]], 1, [action, state], 2, rndseq)
    activated_weights = weights3v2[tiles]

    # transfer
    mapped_target_states = [i for i in range(len(state_mapping)) if state_mapping[i] == int(state)]
    mapped_target_actions = [j for j in range(len(action_mapping)) if action_mapping[j] == int(action)]
    for mapped_target_state in mapped_target_states:
        for mapped_target_action in mapped_target_actions:
            tiles4v3 = GetTiles(num_tilings, ct4v3, [state_val / tile_widthds4v3[int(mapped_target_state)]], 1, [mapped_target_action, mapped_target_state], 2, rndseq)
            transfer_weights[tiles4v3] = copy.deepcopy(activated_weights)
    if row_idx % 500000 == 0:
        print(row_idx)

Transferring weights...
0
500000


In [10]:
for keeper_out in range(1, 5):
    with open('k{}-weights.dat'.format(keeper_out), 'wb') as f:
        for weight in transfer_weights:
            f.write(weight)
        for item in ct4v3.m:
            f.write(item)
        for item in [ct4v3.safe]:
            f.write(item.to_bytes(4, 'little', signed=True))
        for item in [ct4v3.calls]:
            f.write(item.to_bytes(4, 'little', signed=True))
        for item in [ct4v3.clearhits]:
            f.write(item.to_bytes(4, 'little', signed=True))
        for item in [ct4v3.collisions]:
            f.write(item.to_bytes(4, 'little', signed=True))
        for item in ct4v3.data:
            f.write(item)

In [28]:
k3v2_logs_folder = os.path.join(config_data['logs_path'], '202212260013-UbuntuXenial-3v2-4hrs')
k3v2_weights_file = os.path.join(k3v2_logs_folder, 'k1-weights.dat')
k3v2_q_file = os.path.join(k3v2_logs_folder, 'player1672042386.txt')
# new_weights = os.path.join('k1-weights.dat')

In [15]:
qdf = pd.read_csv(k3v2_q_file, index_col = False, names=["State Value", "Action", "State", "Tile"])
# qdf = qdf.drop_duplicates()

In [16]:
len(qdf)

12177984

In [26]:
num_tilings = 32

for row_idx, row in qdf.iterrows():
    state_val, action, state, tile = row['State Value'], row['Action'], row['State'], row['Tile']
    tiles = GetTilesNoCT(num_tilings, [state_val / tile_widthds[row_idx % 32]], 1, [action, state], 2, rndseq)
    if row_idx == 0:
        break

In [27]:
tiles

array([ 278091,   38802,  246765,  512545,  442033,  521545,  541077,
         28506,  864490,  722792,  778303,  322513,  724709,  310299,
        867326,  454540,   67264,  611735,  914773,  326571, 1022982,
        704250,  150427,  766976,  806016,  238271,  630791,  867805,
        749521,   22193,  186460,  846206])

In [29]:
# 3v2
weights3v2, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(k3v2_weights_file)
ct3v2 = collision_table(m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2)
transfer_weights = np.zeros(shape=weights3v2.shape)

In [57]:
k3v2_logs_folder = os.path.join(config_data['logs_path'], '202212260013-UbuntuXenial-3v2-4hrs')
k3v2_weights_file = os.path.join(k3v2_logs_folder, 'k3-weights.dat')
new_weights = os.path.join('k3-weights.dat')
weights3v2, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(k3v2_weights_file)
transfer_weights, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(new_weights)

In [58]:
same = 0
diff = 0
for w1, w2 in zip(weights3v2, transfer_weights):
    if w1 == w2 and w1 != 0.0:
        same += 1
    elif w1 != w2:
        diff += 1
print(same, diff)

12409 113


In [59]:
nonzero_w = 0
for w1 in weights3v2:
    if w1 != 0.0:
        nonzero_w += 1
print(nonzero_w)

12522


In [18]:
num_tilings = 32

for row_idx, row in qdf.iterrows():
    state_val, action, state, tile = row['State Value'], row['Action'], row['State'], row['Tile']
    tiles = GetTiles(num_tilings, ct3v2, [state_val / tile_widthds[int(state)]], 1, [action, state], 2, rndseq)
    activated_weights = weights3v2[tiles]

    # transfer
    transfer_weights[tiles] = copy.deepcopy(activated_weights)
    if row_idx % 500000 == 0:
        print(row_idx)

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
10000000
10500000
11000000
11500000
12000000


In [19]:
with open('k1-weights.dat', 'wb') as f:
    for weight in transfer_weights:
        f.write(weight)
    for item in m3v2:
        f.write(item)
    for item in safe3v2:
        f.write(item)
    for item in calls3v2:
        f.write(item)
    for item in clearhits3v2:
        f.write(item)
    for item in collisions3v2:
        f.write(item)
    for item in data3v2:
        f.write(item)

In [37]:
k3v2_logs_folder = os.path.join(config_data['logs_path'], '202212260013-UbuntuXenial-3v2-4hrs')
k3v2_weights_file = os.path.join(k3v2_logs_folder, 'k2-weights.dat')
k3v2_q_file = os.path.join(k3v2_logs_folder, 'player1672042388.txt')
# new_weights = os.path.join('k1-weights.dat')

In [38]:
qdf = pd.read_csv(k3v2_q_file, index_col = False, names=["State Value", "Action", "State", "Tile"])
qdf = qdf.drop_duplicates(
  subset = ["State Value", "Action", "State"],
  keep = 'first').reset_index(drop = True)
qdf = qdf.dropna()

In [39]:
len(qdf)

309573

In [40]:
# 3v2
weights3v2, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(k3v2_weights_file)
ct3v2 = collision_table(m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2)
transfer_weights = np.zeros(shape=weights3v2.shape)

In [41]:
num_tilings = 32

for row_idx, row in qdf.iterrows():
    state_val, action, state, tile = row['State Value'], row['Action'], row['State'], row['Tile']
    tiles = GetTiles(num_tilings, ct3v2, [state_val / tile_widthds[int(state)]], 1, [action, state], 2, rndseq)
    activated_weights = weights3v2[tiles]

    # transfer
    transfer_weights[tiles] = copy.deepcopy(activated_weights)
    if row_idx % 500000 == 0:
        print(row_idx)

In [42]:
with open('k2-weights.dat', 'wb') as f:
    for weight in transfer_weights:
        f.write(weight)
    for item in m3v2:
        f.write(item)
    for item in safe3v2:
        f.write(item)
    for item in calls3v2:
        f.write(item)
    for item in clearhits3v2:
        f.write(item)
    for item in collisions3v2:
        f.write(item)
    for item in data3v2:
        f.write(item)

In [43]:
k3v2_logs_folder = os.path.join(config_data['logs_path'], '202212260013-UbuntuXenial-3v2-4hrs')
k3v2_weights_file = os.path.join(k3v2_logs_folder, 'k3-weights.dat')
k3v2_q_file = os.path.join(k3v2_logs_folder, 'player1672042390.txt')
# new_weights = os.path.join('k1-weights.dat')

In [44]:
qdf = pd.read_csv(k3v2_q_file, index_col = False, names=["State Value", "Action", "State", "Tile"])
qdf = qdf.drop_duplicates(
  subset = ["State Value", "Action", "State"],
  keep = 'first').reset_index(drop = True)
qdf = qdf.dropna()

In [45]:
len(qdf)

637506

In [46]:
# 3v2
weights3v2, m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2 = parse_weights(k3v2_weights_file)
ct3v2 = collision_table(m3v2, safe3v2, calls3v2, clearhits3v2, collisions3v2, data3v2)
transfer_weights = np.zeros(shape=weights3v2.shape)

In [47]:
num_tilings = 32

for row_idx, row in qdf.iterrows():
    state_val, action, state, tile = row['State Value'], row['Action'], row['State'], row['Tile']
    tiles = GetTiles(num_tilings, ct3v2, [state_val / tile_widthds[int(state)]], 1, [action, state], 2, rndseq)
    activated_weights = weights3v2[tiles]

    # transfer
    transfer_weights[tiles] = copy.deepcopy(activated_weights)
    if row_idx % 500000 == 0:
        print(row_idx)

500000


In [48]:
with open('k3-weights.dat', 'wb') as f:
    for weight in transfer_weights:
        f.write(weight)
    for item in m3v2:
        f.write(item)
    for item in safe3v2:
        f.write(item)
    for item in calls3v2:
        f.write(item)
    for item in clearhits3v2:
        f.write(item)
    for item in collisions3v2:
        f.write(item)
    for item in data3v2:
        f.write(item)